# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    color="City",
    width=1000, 
    height=600,  
    )

#display map plot
map_plot_1


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Filter the city_data_df DataFrame based on the specified conditions
# Filter the city_data_df DataFrame based on the specified conditions
ideal_cities = city_data_df[(city_data_df['Max Temp'] < 27) & 
                            (city_data_df['Max Temp'] > 21) & 
                            (city_data_df['Wind Speed'] < 4.5) & 
                            (city_data_df['Cloudiness'] == 0)]
                         
# Display the cities that meet the criteria
print(ideal_cities)

     City_ID            City      Lat       Lng  Max Temp  Humidity  \
45        45           kapaa  22.0752 -159.3190     22.99        84   
51        51            hilo  19.7297 -155.0900     26.27        83   
63        63           banda  25.4833   80.3333     24.62        52   
81        81   makakilo city  21.3469 -158.0858     21.66        81   
152      152         kahului  20.8947 -156.4700     23.80        60   
197      197             gat  31.6100   34.7642     24.38       100   
211      211          laguna  38.4210 -121.4238     21.67        79   
240      240     tikaitnagar  26.9500   81.5833     23.56        59   
265      265     san quintin  30.4833 -115.9500     21.20        74   
340      340   santa rosalia  27.3167 -112.2833     24.62        56   
363      363          narwar  25.6500   77.9000     22.35        55   
375      375    port hedland -20.3167  118.5667     21.03        73   
381      381       roebourne -20.7833  117.1333     23.48        65   
391   

### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
print(hotel_df.head())

          City Country      Lat       Lng  Humidity Hotel Name
0         faya      SA  18.3851   42.4509        35           
1      farsund      NO  58.0948    6.8047       100           
2  new norfolk      AU -42.7826  147.0587        58           
3    jamestown      US  42.0970  -79.2353        77           
4      lanzhou      CN  36.0564  103.7922        48           


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000 
params = {
    "categories": "accommodation",
    "radius": radius,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{latitude},{longitude},{radius}"
    params["bias"] = f"proximity:{latitude},{longitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
print(hotel_df)

Starting hotel search
faya - nearest hotel: Traktir 333
farsund - nearest hotel: No hotel found
new norfolk - nearest hotel: No hotel found
jamestown - nearest hotel: No hotel found
lanzhou - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
sisimiut - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
colorado - nearest hotel: No hotel found
nikolskoye - nearest hotel: No hotel found
trairi - nearest hotel: No hotel found
dobryanka - nearest hotel: No hotel found
qaanaaq - nearest hotel: No hotel found
mataura - nearest hotel: No hotel found
khatanga - nearest hotel: No hotel found
tasiilaq - nearest hotel: No hotel found
tiksi - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
avarua - nearest hotel: No hotel found
nuevo laredo - nearest hotel: No hotel found
guiratinga - nearest hotel: No hotel found
rikitea - nearest hotel: No hotel 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    tiles="OSM",
    size=10,
    color="City",
    tools=['hover'],
    hover_color='random',
    hover_line_color='black',
    width=1000,
    height=600
)

hotel_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)